In [2]:
import pandas as pd
import os

读取所有品牌信息

In [17]:
os.chdir('/home/carlous/datamining/WebSpider/Brands')
file_path = os.getcwd()

file_csv = []
for root,dirs,files in os.walk(file_path):
    file_csv.append(files)

BrandsData = pd.DataFrame()
for csv in file_csv[0]:
    BrandsData = BrandsData.append(pd.read_csv(csv,header=0,index_col=0,sep=None,),ignore_index=True)

In [44]:
BrandsData.head()

,Third,Brands
0,卫生巾,SOFY/苏菲
1,卫生巾,whisper/护舒宝
2,卫生巾,高洁丝
3,卫生巾,ABC
4,卫生巾,LAURIER/乐而雅


In [38]:
BrandsData.Brands.value_counts()

LOST                             40
BE＆CHEERY/百草味                    38
Three Squirrels/三只松鼠             34
良品铺子                             34
Maryya/美丽雅                       27
MR/妙然                            26
Walch/威露士                        23
禾煜                               23
Nestle/雀巢                        22
妙洁                               22
金龙鱼                              21
十月稻田                             20
edo                              19
Gaia Farm/盖亚农场                   18
茶花                               18
Nivea/妮维雅                        18
盛耳                               18
蓝月亮                              18
PECHOIN/百雀羚                      18
Cleafe/净安                        17
绿伞                               17
隆力奇                              17
燕之坊                              16
洛娃                               16
辉业                               16
SeeSang/鲜享                       16
美农美季                             15
HADAY/海天                    

~~写爬虫太烦了，我决定先不管缺失值了。~~

上句话是用来打脸的

In [45]:
ItemsData = pd.read_csv('../Items.csv',index_col=0)[['Main','title','text']]
ItemsData.rename(columns={'text':'Third'},inplace=True)
ItemsData.head(5)

,Main,title,Third
0,进口食品,进口牛奶,进口纯牛奶
1,进口食品,进口牛奶,进口酸奶
2,进口食品,进口牛奶,进口成人奶粉
3,进口食品,进口牛奶,进口含乳饮料
4,进口食品,进口牛奶,进口有机牛奶


In [47]:
Mergedata = pd.merge(ItemsData,BrandsData,on='Third')

In [98]:
linkData = pd.read_csv('../Items.csv',index_col=0)[['text','productLink']]

temp1 = linkData.rename(columns={'text':'Third'})

temp2 = Mergedata[Mergedata.Brands=='LOST']

LOSTLINK = pd.merge(temp1[['Third','productLink']],temp2[['Third','Brands']],on='Third',how='right')

LOSTLINK.to_csv('LOSTLINK.csv')

In [100]:
import selenium
from selenium import webdriver
import time
import random
import re

def InitChrome():
    options = webdriver.ChromeOptions()
    options.add_argument('lang=zh_CN.UTF-8')
    
    # options.add_argument('user-agent="Mozilla/5.0 (iPod; U; CPU iPhone OS 2_1 like Mac OS X; ja-jp) AppleWebKit/525.18.1 (KHTML, like Gecko) Version/3.1.1 Mobile/5F137 Safari/525.20"')

    browser = webdriver.Chrome(chrome_options=options)
    # browser.get('https://login.tmall.com/?redirectURL=https%3A%2F%2Flist.tmall.com%2F%2Fsearch_product.htm%3Fq%3DThe%2Bland%252F%25C5%25A6%25CA%25CB%25C0%25BC%26user_id%3D725677994%26type%3Dp%26cat%3D50514008%26spm%3Da3204.7933263.a2227oh.d100%26from%3Dchaoshi..pc_1_searchbutton')
    return browser

def getProductLink(arrayLink):
    browser = InitChrome()
    time.sleep(random.randint(2,8))
    browser.get(arrayLink)
    time.sleep(random.randint(2,8))
    print(i)
    try:
        browser.find_element_by_xpath('//*[@id="J_RecommendProp"]/div/a').click()
    except:
        pass
    try:
        temp = browser.find_element_by_xpath('//*[@id="J_RecommendProp"]/div').text
    except:
        print('No value')
        temp = [' ']
    browser.quit()
    return temp

def FeatureEg(name,listToUse):
    # 传入三基变量名称和爬去的变量
    if listToUse!=[' ']:
        RegVal = re.compile(r'\n')
        result = re.split(RegVal,listToUse)
        if len(result)>2:
            temp = pd.DataFrame(result[1:len(result)-2])
        else:
            temp = pd.DataFrame([result[1]]).T
    else:
        temp = pd.DataFrame(listToUse)
    temp['Third']=name
    temp.rename(columns={0:'Brands'},inplace =True)
    return temp[['Third','Brands']]

def ContralFE(sliceIndex,csv_name,listToUse):
    # 传入要保存的三级Item名称（列表）、文件名称、和爬去的数据
    result = FeatureEg(sliceIndex.iloc[0],listToUse[0])
    for i in range(1,len(sliceIndex)):
        print(i,end='-*-')
        try:
            result = result.append(FeatureEg(sliceIndex.iloc[i],listToUse[i]),ignore_index=True)
        except:
            print(csv_name+ ' has error! ')
            pass
    result.to_csv('LOSTLINK/'+csv_name+'.csv')
    print("-*-SAVE"+csv_name)

if __name__ == '__main__':
    
    
    value= pd.read_csv('LOSTLINK.csv',index_col=0)
    
    arrayLink = value.productLink
    sliceIndex = value.Third
    listToUse = []
    for i in arrayLink:
        listToUse.append(getProductLink(i))
    ContralFE(sliceIndex,'LOST.csv',listToUse)

/home/carlous/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


https://list.tmall.com/search_product.htm?spm=1.1.0.0.mg24Hs&user_id=725677994&cat=51462017&active=1&is=brand&acm=lb-zebra-26901-351264.1003.4.468294&style=g&sort=s&scm=1003.4.lb-zebra-26901-351264.OTHER_14440854985286_468294&industryCatId=51462017&brand=8551740
No value


/home/carlous/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


https://list.tmall.com/search_product.htm?spm=1.1.0.0.jdpEn9&user_id=725677994&cat=51450017&active=1&acm=lb-zebra-26901-330370.1003.4.456139&style=g&search_condition=23&scm=1003.4.lb-zebra-26901-330370.OTHER_14434790976547_456139
No value


/home/carlous/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


...

/home/carlous/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


https://list.tmall.com/search_product.htm?cat=51116002&sort=s&style=g&acm=lb-zebra-26901-351367.1003.8.468361&search_condition=23&industryCatId=50514008&active=1&spm=1.1.0.0.braMAI&user_id=725677994&smAreaId=330100&scm=1003.8.lb-zebra-26901-351367.ITEM_14440877728512_468361&tmhkmain=0
No value


/home/carlous/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


https://list.tmall.com/search_product.htm?cat=51120002&sort=s&style=g&acm=lb-zebra-26901-351367.1003.8.468361&search_condition=23&industryCatId=50514008&active=1&spm=1.1.0.0.m4nIBm&user_id=725677994&smAreaId=330100&scm=1003.8.lb-zebra-26901-351367.ITEM_14440889270973_468361&tmhkmain=0
No value


TypeError: expected string or bytes-like object

In [118]:
def FeatureEg(name,listToUse):
    # 传入三基变量名称和爬去的变量
    if listToUse!=[' ']:
        RegVal = re.compile(r'\n')
        result = re.split(RegVal,listToUse)
        if len(result)>2:
            temp = pd.DataFrame(result[1:len(result)-2])
        else:
            temp = pd.DataFrame([result[1]]).T
    else:
        temp = pd.DataFrame(listToUse)
    temp['Third']=name
    temp.rename(columns={0:'Brands'},inplace =True)
    return temp[['Third','Brands']]
def ContralFE(sliceIndex,csv_name,listToUse):
    # 传入要保存的三级Item名称（列表）、文件名称、和爬去的数据
    result = FeatureEg(sliceIndex.iloc[0],listToUse[0])
    for i in range(1,len(sliceIndex)):
        print(i,end='-*-')
        try:
            result = result.append(FeatureEg(sliceIndex.iloc[i],listToUse[i]),ignore_index=True)
        except:
            print(csv_name+ ' has error! ')
            pass
    result.to_csv(csv_name)
    print("-*-SAVE"+csv_name)

ContralFE(sliceIndex,'LOST.csv',listToUse)

1-*-2-*-3-*-4-*-5-*-6-*-7-*-8-*-9-*-10-*-11-*-12-*-13-*-14-*-15-*-16-*-17-*-18-*-19-*-20-*-21-*-22-*-23-*-24-*-25-*-26-*-27-*-28-*-29-*-30-*-31-*-32-*-33-*-34-*-35-*-36-*-37-*-38-*-39-*--*-SAVELOST.csv


In [163]:
Mergedata_2 = pd.read_csv('Brands/LOST.csv',index_col=0)

Result = pd.merge(Mergedata,Mergedata_2,on = ['Third','Brands'],how='left')
SaveValue = Result[Result.Brands!='LOST']

SaveValue.to_csv('Result.csv')
SaveValue.Brands.to_csv('Brands.csv',index=False)